<a href="https://colab.research.google.com/github/Nigam-Niti/FallNet/blob/main/FALL_HMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
cd /content/drive/Shareddrives/Colab_PHD/

/content/drive/Shareddrives/Colab_PHD


In [2]:
# Install dependencies for video module in pytorch to work
! apt-get install libavformat-dev libavdevice-dev
! pip install av==6.2.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libavdevice-dev is already the newest version (7:3.4.8-0ubuntu0.2).
libavformat-dev is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [3]:
cd Fall_pytorch_HMDB51/

/content/drive/Shareddrives/Colab_PHD/Fall_pytorch_HMDB51


#download data hmdb51

In [ ]:
# Download HMDB51 data and splits from serre lab website
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

--2021-12-08 07:18:55--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar [following]
--2021-12-08 07:18:55--  https://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2124008126 (2.0G)
Saving to: ‘hmdb51_org.rar’

hmdb51_org.rar      100%[===================>]   1.98G  31.0MB/s    in 68s     

2021-12-08 07:20:04 (29.6 MB/s) - ‘hmdb51_org.rar’ saved [2124008126/2124008126]

--2021-12-08 07:20:04--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar
R

#Import library

In [4]:

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import torchvision
from torchvision import get_video_backend
from torchvision.models.video import r3d_18 
from torchvision import transforms
import os
from tqdm.auto import tqdm
import numpy as np
import time
import av
import random
print(f"PyAV version -- {av.__version__}")

SEED = 491
torch.manual_seed(SEED)

from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

#run_av_diagnostics()

PyAV version -- 6.2.0


# Extract and organize video data..




In [ ]:
! mkdir -p video_data test_train_splits
! unrar e test_train_splits.rar test_train_splits
! rm test_train_splits.rar
! unrar e hmdb51_org.rar 
! rm hmdb51_org.rar
! mv *.rar video_data
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_data/" + foldername)
    os.system("unrar e video_data/"+ files + " video_data/"+foldername)

! rm video_data/*.rar 


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open test_train_splits.rar
No such file or directory
No files to extract
rm: cannot remove 'test_train_splits.rar': No such file or directory

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open hmdb51_org.rar
No such file or directory
No files to extract
rm: cannot remove 'hmdb51_org.rar': No such file or directory
mv: cannot stat '*.rar': No such file or directory
rm: cannot remove 'video_data/*.rar': No such file or directory


In [ ]:
ls video_data/

brush_hair/    eat/         kiss/        shake_hands/     sword/
cartwheel/     fall_floor/  laugh/       shoot_ball/      sword_exercise/
catch/         fencing/     pick/        shoot_bow/       talk/
chew/          flic_flac/   pour/        shoot_gun/       throw/
clap/          golf/        pullup/      sit/             turn/
climb/         handstand/   punch/       situp/           walk/
climb_stairs/  hit/         push/        smile/           wave/
dive/          hug/         pushup/      smoke/
draw_sword/    jump/        ride_bike/   somersault/
dribble/       kick/        ride_horse/  stand/
drink/         kick_ball/   run/         swing_baseball/


# diagnostics for PyAV installation. For now version 6.2.0 works 


In [5]:
def run_av_diagnostics():
    import av
    av.open("video_data/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ba_goo_4.avi")
    print(get_video_backend())
    av.logging.set_level(av.logging.ERROR)
    if not hasattr(av.video.frame.VideoFrame, 'pict_type'):
      print("Nah")

run_av_diagnostics()

pyav


# memory footprint support libraries/code


In [6]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 87.5 GB  |     Proc size: 1.2 GB
GPU RAM Free: 40536MB | Used: 0MB | Util   0% | Total     40536MB


# download video transformation functions from below link


In [7]:
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
import transforms as T

--2021-12-12 01:24:28--  https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3102 (3.0K) [text/plain]
Saving to: ‘transforms.py.8’

transforms.py.8     100%[===================>]   3.03K  --.-KB/s    in 0.001s  

2021-12-12 01:24:28 (3.83 MB/s) - ‘transforms.py.8’ saved [3102/3102]



#DataLoader_HMDB51
# Datasets and Dataloaders for model training ..


In [8]:

val_split = 0.1
num_frames = 16 # 16
clip_steps = 50
num_workers = 8
pin_memory = True
train_tfms = torchvision.transforms.Compose([
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((400, 400)),
                                 T.RandomHorizontalFlip(),
                                 T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.RandomCrop((224, 224))
                               ])  
test_tfms =  torchvision.transforms.Compose([
                                             T.ToFloatTensorInZeroOne(),
                                             T.Resize((400, 400)),
                                             T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                             T.CenterCrop((224, 224))
                                             ])
hmdb51_train = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=True,
                                                transform=train_tfms, num_workers=num_workers)


hmdb51_test = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=False,
                                                transform=test_tfms, num_workers=num_workers)
      
total_train_samples = len(hmdb51_train)
total_val_samples = round(val_split * total_train_samples)

print(f"number of train samples {total_train_samples}")
print(f"number of validation samples {total_val_samples}")
print(f"number of test samples {len(hmdb51_test)}")


  0%|          | 0/423 [00:00<?, ?it/s]

  0%|          | 0/423 [00:00<?, ?it/s]

number of train samples 7754
number of validation samples 775
number of test samples 3234


#I3d_original

In [9]:
!wget https://raw.githubusercontent.com/piergiaj/pytorch-i3d/master/pytorch_i3d.py
import pytorch_i3d as i3d

--2021-12-12 01:25:15--  https://raw.githubusercontent.com/piergiaj/pytorch-i3d/master/pytorch_i3d.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13801 (13K) [text/plain]
Saving to: ‘pytorch_i3d.py.6’

pytorch_i3d.py.6    100%[===================>]  13.48K  --.-KB/s    in 0.003s  

2021-12-12 01:25:15 (5.20 MB/s) - ‘pytorch_i3d.py.6’ saved [13801/13801]



# OR

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np

import os
import sys
from collections import OrderedDict


class MaxPool3dSamePadding(nn.MaxPool3d):
    
    def compute_pad(self, dim, s):
        if s % self.stride[dim] == 0:
            return max(self.kernel_size[dim] - self.stride[dim], 0)
        else:
            return max(self.kernel_size[dim] - (s % self.stride[dim]), 0)

    def forward(self, x):
        # compute 'same' padding
        (batch, channel, t, h, w) = x.size()
        #print t,h,w
        out_t = np.ceil(float(t) / float(self.stride[0]))
        out_h = np.ceil(float(h) / float(self.stride[1]))
        out_w = np.ceil(float(w) / float(self.stride[2]))
        #print out_t, out_h, out_w
        pad_t = self.compute_pad(0, t)
        pad_h = self.compute_pad(1, h)
        pad_w = self.compute_pad(2, w)
        #print pad_t, pad_h, pad_w

        pad_t_f = pad_t // 2
        pad_t_b = pad_t - pad_t_fimport torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np

import os
import sys
from collections import OrderedDict
…        pad_w = self.compute_pad(2, w)
        #print pad_t, pad_h, pad_w

        pad_t_f = pad_t // 2
        pad_t_b = pad_t - pad_t_f
        pad_h_f = pad_h // 2
        pad_h_b = pad_h - pad_h_f
        pad_w_f = pad_w // 2
        pad_w_b = pad_w - pad_w_f

        pad = (pad_w_f, pad_w_b, pad_h_f, pad_h_b, pad_t_f, pad_t_b)
        #print x.size()
        #print pad
        x = F.pad(x, pad)
        return super().forward(x)
    

class Unit3D(nn.Module):

    def __init__(self, in_channels,
                 output_channels,
                 kernel_shape=(1, 1, 1),
                 stride=(1, 1, 1),
                 padding=0,
                 activation_fn=F.relu,
                 use_batch_norm=True,
                 use_bias=False,
                 name='unit_3d'):
        
        """Initializes Unit3D module."""
        super(Unit3D, self).__init__()
        
        self._output_channels = output_channels
        self._kernel_shape = kernel_shape
        self._stride = stride
        self._use_batch_norm = use_batch_norm
        self._activation_fn = activation_fn
        self._use_bias = use_bias
        self.name = name
        self.padding = padding
        
        self.conv3d = nn.Conv3d(in_channels=in_channels,
                                out_channels=self._output_channels,
                                kernel_size=self._kernel_shape,
                                stride=self._stride,
                                padding=0, # we always want padding to be 0 here. We will dynamically pad based on input size in forward function
                                bias=self._use_bias)
        
        if self._use_batch_norm:
            self.bn = nn.BatchNorm3d(self._output_channels, eps=0.001, momentum=0.01)

    def compute_pad(self, dim, s):
        if s % self._stride[dim] == 0:
            return max(self._kernel_shape[dim] - self._stride[dim], 0)
        else:
            return max(self._kernel_shape[dim] - (s % self._stride[dim]), 0)

            
    def forward(self, x):
        # compute 'same' padding
        (batch, channel, t, h, w) = x.size()
        #print t,h,w
        out_t = np.ceil(float(t) / float(self._stride[0]))
        out_h = np.ceil(float(h) / float(self._stride[1]))
        out_w = np.ceil(float(w) / float(self._stride[2]))
        #print out_t, out_h, out_w
        pad_t = self.compute_pad(0, t)
        pad_h = self.compute_pad(1, h)
        pad_w = self.compute_pad(2, w)
        #print pad_t, pad_h, pad_w

        pad_t_f = pad_t // 2
        pad_t_b = pad_t - pad_t_f
        pad_h_f = pad_h // 2
        pad_h_b = pad_h - pad_h_f
        pad_w_f = pad_w // 2
        pad_w_b = pad_w - pad_w_f

        pad = (pad_w_f, pad_w_b, pad_h_f, pad_h_b, pad_t_f, pad_t_b)
        #print x.size()
        #print pad
        x = F.pad(x, pad)
        #print x.size()        

        x = self.conv3d(x)
        if self._use_batch_norm:
            x = self.bn(x)
        if self._activation_fn is not None:
            x = self._activation_fn(x)
        return x



class InceptionModule(nn.Module):
    def __init__(self, in_channels, out_channels, name):
        super(InceptionModule, self).__init__()

        self.b0 = Unit3D(in_channels=in_channels, output_channels=out_channels[0], kernel_shape=[1, 1, 1], padding=0,
                         name=name+'/Branch_0/Conv3d_0a_1x1')
        self.b1a = Unit3D(in_channels=in_channels, output_channels=out_channels[1], kernel_shape=[1, 1, 1], padding=0,
                          name=name+'/Branch_1/Conv3d_0a_1x1')
        self.b1b = Unit3D(in_channels=out_channels[1], output_channels=out_channels[2], kernel_shape=[3, 3, 3],
                          name=name+'/Branch_1/Conv3d_0b_3x3')
        self.b2a = Unit3D(in_channels=in_channels, output_channels=out_channels[3], kernel_shape=[1, 1, 1], padding=0,
                          name=name+'/Branch_2/Conv3d_0a_1x1')
        self.b2b = Unit3D(in_channels=out_channels[3], output_channels=out_channels[4], kernel_shape=[3, 3, 3],
                          name=name+'/Branch_2/Conv3d_0b_3x3')
        self.b3a = MaxPool3dSamePadding(kernel_size=[3, 3, 3],
                                stride=(1, 1, 1), padding=0)
        self.b3b = Unit3D(in_channels=in_channels, output_channels=out_channels[5], kernel_shape=[1, 1, 1], padding=0,
                          name=name+'/Branch_3/Conv3d_0b_1x1')
        self.name = name

    def forward(self, x):    
        b0 = self.b0(x)
        b1 = self.b1b(self.b1a(x))
        b2 = self.b2b(self.b2a(x))
        b3 = self.b3b(self.b3a(x))
        return torch.cat([b0,b1,b2,b3], dim=1)


class InceptionI3d(nn.Module):
    """Inception-v1 I3D architecture.
    The model is introduced in:
        Quo Vadis, Action Recognition? A New Model and the Kinetics Dataset
        Joao Carreira, Andrew Zisserman
        https://arxiv.org/pdf/1705.07750v1.pdf.
    See also the Inception architecture, introduced in:
        Going deeper with convolutions
        Christian Szegedy, Wei Liu, Yangqing Jia, Pierre Sermanet, Scott Reed,
        Dragomir Anguelov, Dumitru Erhan, Vincent Vanhoucke, Andrew Rabinovich.
        http://arxiv.org/pdf/1409.4842v1.pdf.
    """

    # Endpoints of the model in order. During construction, all the endpoints up
    # to a designated `final_endpoint` are returned in a dictionary as the
    # second return value.
    VALID_ENDPOINTS = (
        'Conv3d_1a_7x7',
        'MaxPool3d_2a_3x3',
        'Conv3d_2b_1x1',
        'Conv3d_2c_3x3',
        'MaxPool3d_3a_3x3',
        'Mixed_3b',
        'Mixed_3c',
        'MaxPool3d_4a_3x3',
        'Mixed_4b',
        'Mixed_4c',
        'Mixed_4d',
        'Mixed_4e',
        'Mixed_4f',
        'MaxPool3d_5a_2x2',
        'Mixed_5b',
        'Mixed_5c',
        'Logits',
        'Predictions',
    )


    def __init__(self, num_classes=400, spatial_squeeze=True,
                 final_endpoint='Logits', name='inception_i3d', in_channels=3, dropout_keep_prob=0.5):
        """Initializes I3D model instance.
        Args:
          num_classes: The number of outputs in the logit layer (default 400, which
              matches the Kinetics dataset).
          spatial_squeeze: Whether to squeeze the spatial dimensions for the logits
              before returning (default True).
          final_endpoint: The model contains many possible endpoints.
              `final_endpoint` specifies the last endpoint for the model to be built
              up to. In addition to the output at `final_endpoint`, all the outputs
              at endpoints up to `final_endpoint` will also be returned, in a
              dictionary. `final_endpoint` must be one of
              InceptionI3d.VALID_ENDPOINTS (default 'Logits').
          name: A string (optional). The name of this module.
        Raises:
          ValueError: if `final_endpoint` is not recognized.
        """

        if final_endpoint not in self.VALID_ENDPOINTS:
            raise ValueError('Unknown final endpoint %s' % final_endpoint)

        super(InceptionI3d, self).__init__()
        self._num_classes = num_classes
        self._spatial_squeeze = spatial_squeeze
        self._final_endpoint = final_endpoint
        self.logits = None

        if self._final_endpoint not in self.VALID_ENDPOINTS:
            raise ValueError('Unknown final endpoint %s' % self._final_endpoint)

        self.end_points = {}
        end_point = 'Conv3d_1a_7x7'
        self.end_points[end_point] = Unit3D(in_channels=in_channels, output_channels=64, kernel_shape=[7, 7, 7],
                                            stride=(2, 2, 2), padding=(3,3,3),  name=name+end_point)
        if self._final_endpoint == end_point: return
        
        end_point = 'MaxPool3d_2a_3x3'
        self.end_points[end_point] = MaxPool3dSamePadding(kernel_size=[1, 3, 3], stride=(1, 2, 2),
                                                             padding=0)
        if self._final_endpoint == end_point: return
        
        end_point = 'Conv3d_2b_1x1'
        self.end_points[end_point] = Unit3D(in_channels=64, output_channels=64, kernel_shape=[1, 1, 1], padding=0,
                                       name=name+end_point)
        if self._final_endpoint == end_point: return
        
        end_point = 'Conv3d_2c_3x3'
        self.end_points[end_point] = Unit3D(in_channels=64, output_channels=192, kernel_shape=[3, 3, 3], padding=1,
                                       name=name+end_point)
        if self._final_endpoint == end_point: return

        end_point = 'MaxPool3d_3a_3x3'
        self.end_points[end_point] = MaxPool3dSamePadding(kernel_size=[1, 3, 3], stride=(1, 2, 2),
                                                             padding=0)
        if self._final_endpoint == end_point: return
        
        end_point = 'Mixed_3b'
        self.end_points[end_point] = InceptionModule(192, [64,96,128,16,32,32], name+end_point)
        if self._final_endpoint == end_point: return

        end_point = 'Mixed_3c'
        self.end_points[end_point] = InceptionModule(256, [128,128,192,32,96,64], name+end_point)
        if self._final_endpoint == end_point: return

        end_point = 'MaxPool3d_4a_3x3'
        self.end_points[end_point] = MaxPool3dSamePadding(kernel_size=[3, 3, 3], stride=(2, 2, 2),
                                                             padding=0)
        if self._final_endpoint == end_point: return

        end_point = 'Mixed_4b'
        self.end_points[end_point] = InceptionModule(128+192+96+64, [192,96,208,16,48,64], name+end_point)
        if self._final_endpoint == end_point: return

        end_point = 'Mixed_4c'
        self.end_points[end_point] = InceptionModule(192+208+48+64, [160,112,224,24,64,64], name+end_point)
        if self._f'inal_endpoint == end_point: return

        end_point = 'Mixed_4d'
        self.end_points[end_point] = InceptionModule(160+224+64+64, [128,128,256,24,64,64], name+end_point)
        if self._final_endpoint == end_point: return

        end_point = 'Mixed_4e'
        self.end_points[end_point] = InceptionModule(128+256+64+64, [112,144,288,32,64,64], name+end_point)
        if self._final_endpoint == end_point: return

        end_point = 'Mixed_4f'
        self.end_points[end_point] = InceptionModule(112+288+64+64, [256,160,320,32,128,128], name+end_point)
        if self._final_endpoint == end_point: return

        end_point = 'MaxPool3d_5a_2x2'
        self.end_points[end_point] = MaxPool3dSamePadding(kernel_size=[2, 2, 2], stride=(2, 2, 2),
                                                             padding=0)
        if self._final_endpoint == end_point: return

        end_point = 'Mixed_5b'
        self.end_points[end_point] = InceptionModule(256+320+128+128, [256,160,320,32,128,128], name+end_point)
        if self._final_endpoint == end_point: return

        end_point = 'Mixed_5c'
        self.end_points[end_point] = InceptionModule(256+320+128+128, [384,192,384,48,128,128], name+end_point)
        if self._final_endpoint == end_point: return

        end_point = 'Logits'
        self.avg_pool = nn.AvgPool3d(kernel_size=[2, 7, 7],
                                     stride=(1, 1, 1))
        self.dropout = nn.Dropout(dropout_keep_prob)
        self.logits = Unit3D(in_channels=384+384+128+128, output_channels=self._num_classes,
                             kernel_shape=[1, 1, 1],
                             padding=0,
                             activation_fn=None,
                             use_batch_norm=False,
                             use_bias=True,
                             name='logits')

        self.build()


    def replace_logits(self, num_classes):
        self._num_classes = num_classes
        self.logits = Unit3D(in_channels=384+384+128+128, output_channels=self._num_classes,
                             kernel_shape=[1, 1, 1],
                             padding=0,
                             activation_fn=None,
                             use_batch_norm=False,
                             use_bias=True,
                             name='logits')
        
    
    def build(self):
        for k in self.end_points.keys():
            self.add_module(k, self.end_points[k])
        
    def forward(self, x):
        for end_point in self.VALID_ENDPOINTS:
            if end_point in self.end_points:
                x = self._modules[end_point](x) # use _modules to work with dataparallel

        x = self.logits(self.dropout(self.avg_pool(x)))
        if self._spatial_squeeze:
            logits = x.squeeze(3).squeeze(3)
        # logits is batch X time X classes, which is what we want to work with
        return logits
        

    def extract_features(self, x):
        for end_point in self.VALID_ENDPOINTS:
            if end_point in self.end_points:
                x = self._modules[end_point](x)
        
        return self.avg_pool(x)

#load pretrained i3d

In [10]:
load_model = 'model/rgb_imagenet.pth'

In [11]:
from torchsummary import summary
mode = 'rgb'
if mode == 'flow':
        i3d = i3d.InceptionI3d(400, in_channels=2)
else:
        i3d = i3d.InceptionI3d(400, in_channels=3)
i3d.load_state_dict(torch.load(load_model))
i3d.replace_logits(51)

i3d.cuda()
summary(i3d, input_size=(3,16,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1      [-1, 64, 8, 112, 112]          65,856
       BatchNorm3d-2      [-1, 64, 8, 112, 112]             128
            Unit3D-3      [-1, 64, 8, 112, 112]               0
MaxPool3dSamePadding-4        [-1, 64, 8, 56, 56]               0
            Conv3d-5        [-1, 64, 8, 56, 56]           4,096
       BatchNorm3d-6        [-1, 64, 8, 56, 56]             128
            Unit3D-7        [-1, 64, 8, 56, 56]               0
            Conv3d-8       [-1, 192, 8, 56, 56]         331,776
       BatchNorm3d-9       [-1, 192, 8, 56, 56]             384
           Unit3D-10       [-1, 192, 8, 56, 56]               0
MaxPool3dSamePadding-11       [-1, 192, 8, 28, 28]               0
           Conv3d-12        [-1, 64, 8, 28, 28]          12,288
      BatchNorm3d-13        [-1, 64, 8, 28, 28]             128
           Unit3D-14        [-1, 6

# Flop count and complexity of i3d

In [12]:
!pip install ptflops

In [13]:
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(i3d, (3,16,224,224), as_strings=False, print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

InceptionI3d(
  12.34 M, 100.000% Params, 27.832 GMac, 100.000% MACs, 
  (avg_pool): AvgPool3d(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, kernel_size=[2, 7, 7], stride=(1, 1, 1), padding=0)
  (dropout): Dropout(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, p=0.5, inplace=False)
  (logits): Unit3D(
    0.052 M, 0.424% Params, 0.0 GMac, 0.000% MACs, 
    (conv3d): Conv3d(0.052 M, 0.424% Params, 0.0 GMac, 0.000% MACs, 1024, 51, kernel_size=(1, 1, 1), stride=(1, 1, 1))
  )
  (Conv3d_1a_7x7): Unit3D(
    0.066 M, 0.535% Params, 6.622 GMac, 23.791% MACs, 
    (conv3d): Conv3d(0.066 M, 0.534% Params, 6.609 GMac, 23.745% MACs, 3, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), bias=False)
    (bn): BatchNorm3d(0.0 M, 0.001% Params, 0.013 GMac, 0.046% MACs, 64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  )
  (MaxPool3d_2a_3x3): MaxPool3dSamePadding(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, kernel_size=[1, 3, 3], stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
 

#save i3d till mixed_5c

In [14]:
i3d_Mixed_5c = torch.nn.Sequential(*list(i3d.children())[3:])

#Sample model

In [ ]:
# set up a sample pytorch class to check if it works
class Model(nn.Module):
  def __init__(self):
      super(Model, self).__init__()
      self.base_model = nn.Sequential(*list(i3d_Mixed_5c.children())[0:])
      self.avg_pool =  nn.AvgPool3d(kernel_size=[1, 7, 7], stride=(1, 1, 1))
      self.relu = nn.ReLU(inplace=True) 
      self.point_conv = nn.Conv1d(in_channels= 1024, out_channels=256, kernel_size=[1], stride=(1))
      self.dropout = nn.Dropout2d(0.3)
      self.fine_conv = nn.Conv1d(256, 2 * 51, kernel_size = 1, stride = 1)
      self.cross_conv = nn.Conv1d(2 * 51,  51, kernel_size = 1, stride = 1)
      self.cross_pool = nn.AvgPool1d(2, stride = 2, padding = 0)                                       
      #self.fc1 = nn.Linear(256, 51)

  def forward(self, x):
      out = self.base_model(x)
      print("size after pretrained model==> ", out.size()) #torch.Size([16, 1024, 2, 7, 7])
      out = self.avg_pool(out) #torch.Size([16, 1024, 2, 1, 1])
      out = self.relu(out).squeeze(-1).squeeze(3) #torch.Size([16, 1024, 2])
      out = self.point_conv(out) #torch.Size([16, 256, 2])
      out = self.dropout(out)
      out = self.fine_conv(out) #torch.Size([16, 102, 2])
      out = self.cross_conv(out) #torch.Size([16, 51, 2])
      out = self.cross_pool(out).squeeze(-1) #torch.Size([16, 51, 1])   
      print("size after cross_pool model==> ", out.size()) #torch.Size([16, 51])
      out = torch.log_softmax(out, dim=1)#torch.Size([16, 51])
      #print(out)
      return out

check = Model().cuda()
torch.cuda.empty_cache()
out = check(torch.randn(16, 3 , 16, 224,224).cuda())
out.size()
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(check, (3,16,224,224), as_strings=False, print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

size after pretrained model==>  torch.Size([16, 1024, 2, 7, 7])
size after cross_pool model==>  torch.Size([16, 51])


torch.Size([16, 51])

#cuda out of memory

In [ ]:
import torch, gc

gc.collect()


88

In [ ]:
torch.cuda.empty_cache()

#Supervision Model_ freeze i3d

In [15]:
cc = 0
for child in i3d_Mixed_5c.children():
  if cc < 16:
    for name, param in i3d_Mixed_5c.named_parameters():
      #  if param.requires_grad and 'Mixed_5b' and 'Mixed_5c'in name:
      param.requires_grad = False
      
    print('child',cc, 'are forzen')
    cc +=1
  else:
    print('child',cc, 'are not forzen')
    cc +=1

child 0 are forzen
child 1 are forzen
child 2 are forzen
child 3 are forzen
child 4 are forzen
child 5 are forzen
child 6 are forzen
child 7 are forzen
child 8 are forzen
child 9 are forzen
child 10 are forzen
child 11 are forzen
child 12 are forzen
child 13 are forzen
child 14 are forzen
child 15 are forzen


In [20]:
del SupervisionNet

In [16]:
class SupervisionNet(nn.Module):
  def __init__(self):
      super(SupervisionNet, self).__init__()
      self.base_model = nn.Sequential(*list(i3d_Mixed_5c.children())[0:])
      self.avg_pool =  nn.AvgPool3d(kernel_size=[1, 7, 7], stride=(1, 1, 1))
      self.relu = nn.ReLU(inplace=True) 
      self.point_conv = nn.Conv1d(in_channels= 1024, out_channels=256, kernel_size=[1], stride=(1))
      self.dropout = nn.Dropout2d(0.3)
      self.fine_conv = nn.Conv1d(256, 2 * 51, kernel_size = 1, stride = 1)
      self.cross_conv = nn.Conv1d(2 * 51,  51, kernel_size = 1, stride = 1)
      self.cross_pool = nn.AvgPool1d(2, stride = 2, padding = 0)                                       
      #self.fc1 = nn.Linear(256, 51)

  def forward(self, x):
      out = self.base_model(x)
      #print("size after pretrained model==> ", out.size()) #torch.Size([16, 1024, 2, 7, 7])
      out = self.avg_pool(out) #torch.Size([16, 1024, 2, 1, 1])
      out = self.relu(out).squeeze(-1).squeeze(3) #torch.Size([16, 1024, 2])
      out = self.point_conv(out) #torch.Size([16, 256, 2])
      out = self.dropout(out)
      out = self.fine_conv(out) #torch.Size([16, 102, 2])
      out = self.cross_conv(out) #torch.Size([16, 51, 2])
      out = self.cross_pool(out).squeeze(-1) #torch.Size([16, 51, 1])   
      #print("size after cross_pool model==> ", out.size()) #torch.Size([16, 51])
      out = torch.log_softmax(out, dim=1)#torch.Size([16, 51])
      #print(out)
      return out

In [ ]:
check = SupervisionNet().cuda()
torch.cuda.empty_cache()
out = check(torch.randn(16, 3 , 16, 224,224).cuda())
out.size()

In [17]:
check = SupervisionNet().cuda()
from ptflops import get_model_complexity_info

macs, params = get_model_complexity_info(check, (3,16,224,224), as_strings=False, print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

SupervisionNet(
  0.294 M, 100.000% Params, 27.833 GMac, 100.000% MACs, 
  (base_model): Sequential(
    0.0 M, 0.000% Params, 27.832 GMac, 99.998% MACs, 
    (0): Unit3D(
      0.0 M, 0.000% Params, 6.622 GMac, 23.791% MACs, 
      (conv3d): Conv3d(0.0 M, 0.000% Params, 6.609 GMac, 23.745% MACs, 3, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), bias=False)
      (bn): BatchNorm3d(0.0 M, 0.000% Params, 0.013 GMac, 0.046% MACs, 64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
    (1): MaxPool3dSamePadding(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, kernel_size=[1, 3, 3], stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
    (2): Unit3D(
      0.0 M, 0.000% Params, 0.106 GMac, 0.381% MACs, 
      (conv3d): Conv3d(0.0 M, 0.000% Params, 0.103 GMac, 0.369% MACs, 64, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn): BatchNorm3d(0.0 M, 0.000% Params, 0.003 GMac, 0.012% MACs, 64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True

#AverageMeter

In [18]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def update_acc(self, val, n=1):
        self.val = val/n
        #print('val',self.val)
        self.sum += val
        #print('sum',self.sum)
        self.count += n
        #print('count',self.count)
        self.avg = self.sum / self.count
        #print('avg',self.avg)

#save check points


In [23]:
import shutil

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

#Train

In [20]:
def train(config, model, loader, optimizer, epoch, checkpoint_path, best_model_path):
    print('#'*60)
    print('Epoch {}. Starting with training phase.'.format(epoch))
    model.train()
    config = {}
    config['log_interval'] = 100
    correct = 0
    total_loss = 0.0
    flag = 0
    Loss, Acc = AverageMeter(), AverageMeter()
    start = time.time()
    for batch_id, data in enumerate(loader):
        data, target = data[0], data[-1]
        # print("here")

        if torch.cuda.is_available():
           data = data.cuda()
           target = target.cuda()

        optimizer.zero_grad()
        output = model(data)
        #loss = F.nll_loss(output, target)
        loss = nn.CrossEntropyLoss()(output, target)
        #loss = WFocalLoss()(output, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        Loss.update(loss.item(), data.size(0))

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        num_corrects = pred.eq(target.view_as(pred)).sum().item()
        correct += num_corrects

        Acc.update_acc(num_corrects, data.size(0))

        if flag!= 0 and batch_id%config['log_interval'] == 0:
           print('Train Epoch: {} Batch [{}/{} ({:.0f}%)]\tLoss: {:.6f} Accuracy: {}/{} ({:.0f})%'.format(
                epoch, batch_id * len(data), len(loader.dataset),
                100. * batch_id / len(loader), Loss.avg, correct, Acc.count, 100. * Acc.avg))
        flag = 1

        
        # create checkpoint variable and add important data
        checkpoint = {
            'epoch': epoch,
            'loss_min': Loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict(),
        }
        
        # save checkpoint
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        

    #total_loss /= len(loader.dataset) 
    print('Train Epoch: {} Average Loss: {:.6f} Average Accuracy: {}/{} ({:.0f})%'.format(
         epoch, Loss.avg, correct, Acc.count, 100. * Acc.avg ))
    print(f"Takes {time.time() - start}")

#Test

In [21]:
def test(config, model, loader, text='Validation'):
    model.eval()
    correct = 0
    total_loss = 0.0
    Loss, Acc = AverageMeter(), AverageMeter()
    with torch.no_grad():
         for batch_id, data in enumerate(loader):
             data, target = data[0], data[-1]

             if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()

             output = model(data)
             #loss = F.nll_loss(output, target)
             loss = nn.CrossEntropyLoss()(output, target)
             #loss = WFocalLoss()(output, target)

             total_loss += loss.item()

             Loss.update(loss.item(), data.size(0))

             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
             num_corrects = pred.eq(target.view_as(pred)).sum().item()
             correct += num_corrects

             Acc.update_acc(num_corrects, data.size(0))
           
    total_loss /= len(loader.dataset)
    print(text + ' Average Loss: {:.6f} Average Accuracy: {}/{} ({:.0f})%'.format(
         Loss.avg, correct, Acc.count , 100. * Acc.avg ))

#main

In [ ]:

bs = 16
lr = 1e-4
gamma = 0.6
total_epochs = 100
config = {}
num_workers = 1

kwargs = {'num_workers':num_workers, 'pin_memory':True} if torch.cuda.is_available() else {'num_workers':num_workers}
#kwargs = {'num_workers':num_workers}
#kwargs = {}

hmdb51_train_v1, hmdb51_val_v1 = random_split(hmdb51_train, [total_train_samples - total_val_samples,
                                                                       total_val_samples])

#hmdb51_train_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_val_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_test.video_clips.compute_clips(16, 1, frame_rate=30)

#train_sampler = RandomClipSampler(hmdb51_train_v1.video_clips, 5)
#test_sampler = UniformClipSampler(hmdb51_test.video_clips, 5)
 #AssertionError: torch.Size([17, 240, 320, 3]) x 16 
train_loader = DataLoader(hmdb51_train_v1, batch_size=bs, shuffle=True, **kwargs)
val_loader   = DataLoader(hmdb51_val_v1, batch_size=bs, shuffle=True, **kwargs)
test_loader  = DataLoader(hmdb51_test, batch_size=bs, shuffle=False, **kwargs)

model = SupervisionNet()

if torch.cuda.is_available():
   model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=gamma)

print("Launching Action Recognition Model training")
for epoch in range(1, total_epochs + 1):
    #print(train_loader)
    train(config, model, train_loader, optimizer, epoch, 'save_model/current_ckpt_cc_0.4.pth','best_path/best_model_cc_0.4.pth')
    test(config, model, val_loader, text="Validation")
    scheduler.step()

test(config, model, test_loader, text="Test")

Launching Action Recognition Model training
############################################################
Epoch 1. Starting with training phase.
Train Epoch: 1 Batch [1600/6979 (23%)]	Loss: 3.882209 Accuracy: 107/1616 (7)%
Train Epoch: 1 Batch [3200/6979 (46%)]	Loss: 3.829464 Accuracy: 264/3216 (8)%
Train Epoch: 1 Batch [4800/6979 (69%)]	Loss: 3.792506 Accuracy: 472/4816 (10)%
Train Epoch: 1 Batch [6400/6979 (92%)]	Loss: 3.741581 Accuracy: 759/6416 (12)%
Train Epoch: 1 Average Loss: 3.717413 Average Accuracy: 881/6979 (13)%
Takes 870.7088949680328
Validation Average Loss: 3.396286 Average Accuracy: 175/775 (23)%
############################################################
Epoch 2. Starting with training phase.
Train Epoch: 2 Batch [1600/6979 (23%)]	Loss: 3.325147 Accuracy: 356/1616 (22)%
Train Epoch: 2 Batch [3200/6979 (46%)]	Loss: 3.246881 Accuracy: 755/3216 (23)%
Train Epoch: 2 Batch [4800/6979 (69%)]	Loss: 3.150639 Accuracy: 1269/4816 (26)%
Train Epoch: 2 Batch [6400/6979 (92%)]	Loss